In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from math import sqrt

from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Activation, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
#Run Local
#df = pd.read_csv("csv/book_info_complete.csv")
#Run on COLAB
df = pd.read_csv("/content/drive/My Drive/final_project/book_info_complete.csv")

In [5]:
df = df.dropna(axis=1)
df["Publication date"] = df["Publication date"].str[-4:].astype(int)
df = df.drop("Last updated", axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5816 entries, 0 to 5815
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             5816 non-null   object
 1   description       5816 non-null   object
 2   plot              5816 non-null   object
 3   csm_review        5816 non-null   object
 4   need_to_know      5816 non-null   object
 5   csm_rating        5816 non-null   int64 
 6   Genre             5816 non-null   object
 7   Book type         5816 non-null   object
 8   Publication date  5816 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 409.1+ KB


In [ ]:
#df["text"] = df['title'] + " " + df['plot'] + " " + df["description"] + " " + df["csm_review"] + " " + df["need_to_know"]

## Create the splits

In [ ]:
def splitter(df):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=999)
    for train_index, test_index in split.split(df, df['csm_rating']):
        train_data= df.loc[train_index]
        test_data = df.loc[test_index]
    
    
    return train_data, test_data

In [ ]:
train_data, test_data = splitter(df)

In [ ]:
x_tr, y_tr = train_data['csm_review'].values, train_data["csm_rating"].values
x_val, y_val = test_data["csm_review"].values, test_data["csm_rating"].values

In [10]:
print(x_tr.shape, y_tr.shape)

(4652,) (4652,)


In [11]:
print(x_val.shape, y_val.shape)

(1164,) (1164,)


## Prepare the Data

In [12]:
#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

print(len(max(x_tr_seq, key=len)))
max_length = len(max(x_tr_seq, key=len))
print(len(min(x_tr_seq, key=len)))

502
17


In [13]:
#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=max_length)
x_val_seq = pad_sequences(x_val_seq, maxlen=max_length)
print(len(max(x_tr_seq, key=len)))
print(len(min(x_tr_seq, key=len)))

502
502


In [14]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

30700


## Create embeddings

In [15]:
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

Found 30699 unique tokens.


## Create the Model

In [16]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=max_length,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='relu')) 

#Add loss function, metrics, optimizer
#optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(optimizer="RMSprop", loss='mean_squared_error', metrics=["mae"]) 

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 502, 300)          9210000   
_________________________________________________________________
lstm (LSTM)                  (None, 502, 128)          219648    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 9,437,969
Trainable params: 9,437,969
Non-trainable params: 0
_________________________________________________________________
None


## Callbacks

In [ ]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

## Fit the Model

In [18]:
history = model.fit(np.array(x_tr_seq),
                    np.array(y_tr),
                    batch_size=128,
                    epochs=1000,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/1000
37/37 [==============================] - 6s 167ms/step - loss: 23.8846 - mae: 3.9629 - val_loss: 15.2217 - val_mae: 3.3077 - lr: 0.0010
Epoch 2/1000
37/37 [==============================] - 6s 151ms/step - loss: 15.1336 - mae: 3.2852 - val_loss: 14.8669 - val_mae: 3.2701 - lr: 0.0010
Epoch 3/1000
37/37 [==============================] - 6s 153ms/step - loss: 13.5917 - mae: 3.1083 - val_loss: 12.3484 - val_mae: 2.9404 - lr: 0.0010
Epoch 4/1000
37/37 [==============================] - 6s 151ms/step - loss: 10.0877 - mae: 2.6761 - val_loss: 8.9303 - val_mae: 2.4490 - lr: 0.0010
Epoch 5/1000
37/37 [==============================] - 6s 151ms/step - loss: 6.7190 - mae: 2.1270 - val_loss: 7.1058 - val_mae: 2.0478 - lr: 0.0010
Epoch 6/1000
37/37 [==============================] - 6s 152ms/step - loss: 5.0808 - mae: 1.7859 - val_loss: 6.7963 - val_mae: 1.9484 - lr: 0.0010
Epoch 7/1000
37/37 [==============================] - 6s 150ms/step - loss: 4.2072 - mae: 1.5926 - val_loss: 5.

In [21]:
#evaluation 
val_loss, val_mae = model.evaluate(x_val_seq, y_val)

print("The val_mae is %.3f." % val_mae)

37/37 [==============================] - 1s 18ms/step - loss: 4.7706 - mae: 1.6826
The val_mae is 1.683.


In [23]:
plt.plot(model.history.history["loss"], label="loss");
plt.plot(model.history.history["val_loss"], label="val_loss");
plt.legend();
plt.show();
plt.close();

plt.plot(model.history.history["mae"], label="mae");
plt.plot(model.history.history["val_mae"], label="val_mae");
plt.legend();

KeyError: ignored

## [Use Transfer Learning](https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/)

In [24]:
# load the whole embedding into memory
embeddings_index = dict()

with open("/content/drive/My Drive/Colab Notebooks/glove.6B/glove.6B.300d.txt") as f:

    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
model2=Sequential()

#embedding layer
model2.add(Embedding(size_of_vocabulary,300,input_length=max_length,trainable=True)) 

#lstm layer
model2.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model2.add(GlobalMaxPooling1D())

#Dense Layer
model2.add(Dense(64,activation='relu')) 
model2.add(Dense(1,activation='relu')) 

#Add loss function, metrics, optimizer
#optimizer = tf.keras.optimizers.RMSprop(0.001)

model2.compile(optimizer="RMSprop", loss='mean_squared_error', metrics=["mae"]) 

#Print summary of model
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 502, 300)          9210000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 502, 128)          219648    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 9,437,969
Trainable params: 9,437,969
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
history = model2.fit(np.array(x_tr_seq),
                    np.array(y_tr),
                    batch_size=128,
                    epochs=1000,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/1000
37/37 [==============================] - 6s 162ms/step - loss: 23.7724 - mae: 3.9680 - val_loss: 15.4418 - val_mae: 3.3052 - lr: 0.0010
Epoch 2/1000
37/37 [==============================] - 6s 151ms/step - loss: 14.8373 - mae: 3.2595 - val_loss: 14.2261 - val_mae: 3.1865 - lr: 0.0010
Epoch 3/1000
37/37 [==============================] - 6s 155ms/step - loss: 13.0930 - mae: 3.0567 - val_loss: 12.3030 - val_mae: 2.9578 - lr: 0.0010
Epoch 4/1000
37/37 [==============================] - 6s 153ms/step - loss: 10.2579 - mae: 2.6864 - val_loss: 8.8424 - val_mae: 2.4816 - lr: 0.0010
Epoch 5/1000
37/37 [==============================] - 6s 153ms/step - loss: 7.1606 - mae: 2.1961 - val_loss: 6.5234 - val_mae: 2.0599 - lr: 0.0010
Epoch 6/1000
37/37 [==============================] - 6s 154ms/step - loss: 5.1792 - mae: 1.8175 - val_loss: 5.8048 - val_mae: 1.9147 - lr: 0.0010
Epoch 7/1000
37/37 [==============================] - 6s 153ms/step - loss: 4.1118 - mae: 1.5719 - val_loss: 5.

In [28]:
#evaluation 
_, val_mae = model2.evaluate(x_val_seq, y_val)

print("The val_mae is %.3f." % val_mae)

37/37 [==============================] - 1s 19ms/step - loss: 4.9926 - mae: 1.7202
The val_mae is 1.720.


In [29]:
model2.save('/content/drive/My Drive/final_project/lstm_csm_review_transfer_model_scaled')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


FailedPreconditionError: ignored

In [31]:
plt.plot(model2.history.history["loss"], label="loss");
plt.plot(model2.history.history["val_loss"], label="val_loss");
plt.legend();
plt.show();
plt.close();

plt.plot(model2.history.history["mae"], label="mae");
plt.plot(model2.history.history["val_mae"], label="val_mae");
plt.legend();

KeyError: ignored

In [ ]:
a = model2.predict(x_val_seq)
b = model2.predict(x_val_seq)
#a = min_max_scaler_target.inverse_transform(a)
#b = min_max_scaler_target.inverse_transform(b)

csm_lstm_predictions_df = pd.DataFrame({"csm_custom": list(a), "csm_transfer": list(b)},
                              columns = ["csm_custom", "csm_transfer"], 
                              index=test_data.index)

csm_lstm_predictions_df.to_csv('/content/drive/My Drive/final_project/lstm_csm_review.csv', index="False")
